In [1]:
import cobra as cb
import pandas as pd
import re

In [2]:
m = cb.io.read_sbml_model('iSR432.xml')
print('iSR432 exchanges:', m.exchanges) # Exchange reactions are not included in the sbml file...
bounds = pd.read_csv('iSR432_exchanges.tsv', '\t')
bounds.head()

iSR432 exchanges: []


,ID,lb,ub
0,ca2[e],-1000.0,0.0
1,cellulose[e],-1000.0,0.0
2,biomass[c],0.0,1000.0
3,no2[e],0.0,1000.0
4,fru[e],-1000.0,0.0


In [3]:
def update_notation(met_id):
    # Cobrapy changes name with respect to the table above
    met_id = str.replace(met_id,'-','_DASH_')
    met_id = str.replace(met_id, '_l', '_L')
    met_id = str.replace(met_id, '_d', '_D')
    met_id = str.replace(met_id, '_r', '_R')
    return met_id[:-3] + '_' + re.findall(r'.*\[([c,e])\]', met_id)[0]
    
for index, row in bounds.iterrows():
    met_id = update_notation(row['ID'])
    if met_id in m.metabolites:
        met = m.metabolites.get_by_id(met_id)
    elif (met_id[:-2] + '_c') in m.metabolites:
        met = m.metabolites.get_by_id(met_id[:-2] + '_c') 
    else:
        print('Metabolite {} not in model'.format(update_notation(row['ID']) ))
        continue
    m.add_boundary(met, lb=row['lb'], ub=row['ub'])

    #m.add_boundary(met, lb=-1000, ub=1000) # This way the model can grow
m.objective = m.reactions.biomass_target
m.optimize()
m.summary()

Metabolite biomass_c not in model
Metabolite no2_e not in model
Metabolite cellmass_c not in model
IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  -----------------
                           biomass_target  0


The three missing metabolites are not in the xml model at all. The two tentative cell mass pseudo-metabolites are not in the biomass reaction.
If the -1000,1000 exchanges are used instead of those provided by the authors, then the model can grow. This indicates that an essential metabolite is missing form the input, or that secretion of a certain metabolite must be enabled.

In [7]:
cb.io.save_json_model(m, 'iSR432_w_exch.json')